## CS431/631 Big Data Infrastructure
### Winter 2018 - Assignment 5
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Yangjie_
* **ID:** _20744733_

---
#### Overview
For this assignment, you will be using Python and Spark to perform spam detection.   You will need to perform two tasks.   The first is to build spam prediction models, using training data sets and stochastic gradient descent (SGD).   The second is to use these models to predict whether the documents in a test data set are spam.
The stochastic gradient descent technique that you will be using is based on [a paper](http://arxiv.org/abs/1004.5168) by Cormack, Smucker and Clarke.

#### Training a Spam Classification Models
To build a spam classification model, you will start with a training data set.   Each instance in the training set represents a single document, and is labeled to indicate whether that document should be considered to be spam or ham.
An instance looks like this:
```
clueweb09-en0094-20-13546 spam 387908 697162 426572 161118 688171 ...
```
The first field, `clueweb09-en0094-20-13546`, is the (unique) document name.   The second field is the label, indicating whether the document should be considered spam (as in this example) or ham.   The remaining fields are integers representing *features* present in the document.   In this case, the features are hashed byte 4-grams, represented as integers.   Each training data set is stored as a text file, with one instance per line.   The training files have been preloaded into the directory `/u/cs451/public_html/spam` for you.   They are:
* `spam.train.group_x.txt`   (25 MB)
* `spam.train.group_y.txt`   (20 MB)
* `spam.train.britney.txt`   (766 MB)

#### Question 1 ( 5/20 marks)

Your first task is to write a sequential SGD model trainer in Python (no Spark).   For our purposes, a model associates a *weight* with each feature.   The model trainer decides what these weights should be, based on the training instances.  Since you are going to be writing a model trainer based on SGD, the trainer should behave like this:
```
for each training instance T
   predict whether T is spam or ham using the weights of the current model
   update the model weights by comparing T's predicted label with its actual label
```
Of course, the important part is how to update the model.

In [the paper](http://arxiv.org/abs/1004.5168), the model is used to assign a "spamminess" score to a document.   Documents with positive spamminess are predicted to be spam.   Those with negative spamminess are predicted to be ham.  The spamminess of a document $D$ is simply the sum of the weights (from the model) of each of the document's features:
\begin{equation}
spamminess(D) = \sum_{f \in D}{w(f)}
\end{equation}
where $w(f)$ is the weight assocated with feature $f$.

The Python module `spamminess.py` defines a function `spamminess(F,W)` which computes this quantity.   This function takes two arguments, `F` and `W`.  `F` is a list of features (integers) associated the document whose spamminess you want to compute, and `W` is a dictionary representing the current model.  `W` maps features ($f$) to their weights ($w(f)$) under the model.

In the cell below, you will find partial pseudo-code that shows how to implement the SGD model trainer defined by Cormack, Smucker, and Clarke.   It reads the training instances one at a time from one of the training files, and uses them to adjust then model weights.   You job is to turn this pseudo-code into actual runnable Python code that can
be used to learn a model from any one of the training files.   Once your code has finished reading and processing all of the instances from the input training file, your code should print the 5 features with the highest scores and the five features with the lowest scores in the model you have learned.   For each of these 10 features, print both the feature and its score.


In [1]:
from spamminess import spamminess
from math import exp
import re

w = {}
delta = 0.002

with open('/u/cs451/public_html/spam/spam.train.group_x.txt') as f:
    for line in f:
        D=line.strip().split(' ')
        t=D[1]
        F=set(D[2:])
        
#       find the spamminess of the current document using the current model:
        score=spamminess(F,w)
    
#       Update the model:
        prob=1.0/(1+exp(-score))
        for k in F:
            if t=="spam":
                if k not in w:
                    w[k]=(1.0-prob)*delta
                else:
                    w[k]+=(1.0-prob)*delta
            elif t=="ham":
                if k not in w:
                    w[k]=-prob*delta
                else:
                    w[k]-=prob*delta
                    
q1_result = sorted(zip(w.values(),w.keys()),reverse=True)
print('5 features with the highest scores is:')
for i in q1_result[0:5]:
    print('featrue:{0} \t score:{1}'.format(i[1],i[0]))


print('\n5 features with the lowest scores is:')
for j in q1_result[-5:]:
    print('featrue:{0} \t score:{1}'.format(j[1],j[0]))

5 features with the highest scores is:
featrue:288281 	 score:0.022996007768337445
featrue:316070 	 score:0.021787607689746986
featrue:305568 	 score:0.021669913959345807
featrue:737304 	 score:0.02155627086676941
featrue:102264 	 score:0.02138172380411256

5 features with the lowest scores is:
featrue:646100 	 score:-0.021927190429925532
featrue:253515 	 score:-0.021992394685373975
featrue:402756 	 score:-0.02230073520257498
featrue:585043 	 score:-0.023067846652662174
featrue:358032 	 score:-0.024399648385718772


#### Question 2 (5/20 marks)

Next, you should try implementing a Spark version of the SGD model trainer.   Before you start, create a new folder
called `models` inside your cs431 folder.   Your Spark implementation should read a training file, train the model, and then output the model to the models folder.   For each of the three training sets, you should produce a different model.  The model output file that you generate should list the weight associated with each feature, with one feature per line, like this:
```
(802123, 0.0009858585991850937)
(438450, 4.267897922108138e-05)
(271525, 0.0013133437007968654)
(92853, 0.0004300009932503611)
```

Use Spark's `saveAsTextFile` action to output your model.   For example, if you are training a model for the group_x training set, use `saveAsTextFile("models/group_x_model")`.   This will actually cause Spark to create folder called `group_x_model`.   In the folder, there will be files with names like `part-00000` that contain the actual output data.  When you use `saveAsTextFile`, Spark will generate one `part-xxxx` file for each partition of the RDD that you are writing out.   In this case, you should have only a single partition (for the reason described below), so there should be only one `part-xxxx` file.

Training the SGD model is an inherently sequential task, since the training instances update the model one at a time, and each instance's spamminess is computed using the model produced by that instance's predecessors.   This means that the only part of the training that you can parallelize using Spark is the parsing of the input file.   Once the input is parsed, your Spark implementation will have to force all of the instances into a single partition, and then apply the training function to the entire partition.   To see whether you are getting sensible results, you can compare the model you learn with Spark to the one that you learned with your sequential Python program from Question 1.

Remember that training should occur entirely in Spark.  The training instances should never come into your driver program.

Put your solution for Question 2 into the cell below, and use it to generate models from all three of the training files, leaving the results in your models folder.   For this assignment, it is more natural to use Spark's original RDD interface, rather than the DataFrame interface (why?).   The Spark initialization code in the cell below assumes that you will be using the RDD interface.


In [2]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="YourTest", master="local[2]")
from spamminess import spamminess
from math import exp

def fun(iterator):
    for x in iterator:
        D = x[1].strip().split(' ')
        F = list(set(D[2:]))
        t = D[1]
        score = spamminess(F, w)
        prob = 1.0 / (1 + exp(-score))
        for f in F:
            if t == "spam":
                if f not in w:
                    w[f] = (1.0 - prob) * delta
                else:
                    w[f] += (1.0 - prob) * delta
            elif t == "ham":
                if f not in w:
                    w[f] = -prob * delta
                else:
                    w[f] -= prob * delta
    return list(w.items())

delta = 0.002
txt_names = ['spam.train.group_x','spam.train.group_y','spam.train.britney']
file_names = ['group_x_model','group_y_model','britney_model']

for i in range(3):
    w={}
    row_RDD = sc.textFile('/u/cs451/public_html/spam/{0}.txt'.format(txt_names[i]),1)
    
    nrow = row_RDD.count()
    index = iter([x for x in range(nrow)])
    document_RDD = row_RDD.map(lambda x: (next(index), x))
    weight_RDD=document_RDD.mapPartitions(fun)\
            .saveAsTextFile('/u0/y554zhou/cs631/Assignment5/models/{0}'.format(file_names[i]))
print('Done!')
    

Done!


#### Question 3 (5/20 marks)

When you train a model using SGD, the model you get depends on the order in which you handle the training instances.  To see this in action, try using the Spark SGD trainer you implemented for Question 2 to train a model from the group_x training set, but with the instances processed in a different order.  

To do this, modify your trainer from Question 2 so that it will randomly reorder the training instances before using them to update the model.   One way to shuffle the training instances is to assign a random sort key to each training instance as you read it from the input file, and then sort the instances using the random sort key.

Be sure that Spark is doing the work of shuffling the training instances.   Do not load the training instances into your driver program and sort them there.

Once you have implemented the shuffled trainer, train a model using shuffled group_x training instances, and compare the resulting model with group_x model you learned without shuffling.   It is up to you how to do this comparision.  At a minimum, compare features with the highest weights in each model to see if they are similar.


In [3]:
#  Put your shuffling Spark SGD trainer in this cell.
#  In addition, add one or more new cells (code and/or text) below this one to 
#  compare the shuffled and unshuffled models you produced for group_x


from spamminess import spamminess
from math import exp
import random

# [using the random seed 12345]
random.seed(12345)

def fun(iterator):
    for x in iterator:
        D = x[1].strip().split(' ')
        F = list(set(D[2:]))
        t = D[1]
        score = spamminess(F, w)
        prob = 1.0 / (1 + exp(-score))
        for f in F:
            if t == "spam":
                if f not in w:
                    w[f] = (1.0 - prob) * delta
                else:
                    w[f] += (1.0 - prob) * delta
            elif t == "ham":
                if f not in w:
                    w[f] = -prob * delta
                else:
                    w[f] -= prob * delta
    return list(w.items())

delta = 0.002
txt_names = ['spam.train.group_x']

for i in range(1):
    w={}
    row_RDD = sc.textFile('/u/cs451/public_html/spam/{0}.txt'.format(txt_names[i]),1).persist()

    nrow = row_RDD.count()
    # using random sample to reorder the instances
    index = iter(random.sample([x for x in range(nrow)],nrow))
    document_RDD = row_RDD.map(lambda x: (next(index), x)).sortByKey()
    weight_RDD=document_RDD.mapPartitions(fun).sortBy(lambda x: x[1],ascending=False).take(5)
    print('the top 5 hightest features with their weights in shuffled model')
    for j in weight_RDD:
        print(j)
    print('the top 5 hightest features with their weights in unshuffled model')
    for j in q1_result[0:5]:
        print((j[1],j[0]))
print('it is quite clear that both the highest featues and their weights are different')
    
    
    
    

the top 5 hightest features with their weights in shuffled model
('293364', 0.026729745410971534)
('658098', 0.026729745410971534)
('419514', 0.024541677025685264)
('837735', 0.02311132572083204)
('520474', 0.02277934338579715)
the top 5 hightest features with their weights in unshuffled model
('288281', 0.022996007768337445)
('316070', 0.021787607689746986)
('305568', 0.021669913959345807)
('737304', 0.02155627086676941)
('102264', 0.02138172380411256)
it is quite clear that both the highest featues and their weights are different


#### Question 4 (5/20  marks)

Last but not least, you should write a Spark program that can be used to classify documents as spam or ham, using the classification models you produced.

The test data, i.e., the document instances that you should classifiy, is located in the file
* `/u/cs451/public_html/spam/spam.test.qrels.txt`

Each line in this file represents a document that needs to be classified as spam or ham.  The format of this file is identical to the format of the files that hold the training instances.

Write a Spark program that will load a model (from a specified folder under `models`), classify all of the instances in `spam.test.qrels.txt` using that model, and then output the results as a text file.   The contents of the output file should look like this:
```
(clueweb09-en0000-00-00142,spam,2.601624279252943,spam)
(clueweb09-en0000-00-01005,ham,2.5654162439491004,spam)
(clueweb09-en0000-00-01382,ham,2.5893946346394188,spam)
```
Each line of the output represents one test instance.   The first two fields are the document ID and the test label.  These are just copied from the test data.   The third field is the spamminess score of the document, produced by the spamminess function using the model you are classifying with.   The fourth field is the spam/ham prediction made by the model.

Of course, your spam/ham classifier must **not** use the test label from the input when making its prediction.  The test labels are the "ground truth" against which your predictions are being compared.   Using them to make predictions would defeat the whole purpose of model-based classification.

Make sure that classification of the test instances is done by Spark, not by your driver program.  Do ***not*** load the test instances or classification results into your driver program.
dUnlike model training, classification is easily parallelizable, since each document is classified independently.   

In [4]:
def fun3(x):
    D = x.strip().split(' ')
    F = list(set(D[2:]))
    t = D[1]
    ID = D[0]
    score = spamminess(F, w)
    if score>0:
        return (ID,t,score,'spam')
    else:
        return (ID,t,score,'ham')

    
file_names = ['group_x_model','group_y_model','britney_model']
for i in range(3): 
    model=sc.textFile('/u0/y554zhou/cs631/Assignment5/models/{0}/part-00000'.format(file_names[i]))\
            .map(lambda x:eval(x))\
            .map(lambda x: (1, x))\
            .groupByKey()\
            .mapValues(lambda x: sorted(x)).collect()
    w=dict(model[0][1])

    test=sc.textFile('/u/cs451/public_html/spam/spam.test.qrels.txt')\
            .map(fun3).saveAsTextFile('/u0/y554zhou/cs631/Assignment5/output-file/{0}'.format(file_names[i]))
    

We have installed a program in the CS451 account that can be used to evaluate your classification results.  Given your ouput file, in the proper format, it will compute the area under the receiver operating curve (ROC).   This is a common way to characterize classifier error.    The lower this score, the better.   The evaluation program should produce one line of output, like this
```
1-ROCA%: 17.25
```

Use your classifier to classify the test instances using each of the three classification models that you produced, which should result in three different output files.   Then, in the cell below,
use the evaluation program to evaluate each of the three results.

In [5]:
#  Run the evaluation program like this, after first replacing "output-file"
#  with the name of the file that holds your classifier's output
#  Note the "!" character, which is important.   This is the escape character
#  that tells the notebook to run an external program.

!/u/cs451/bin/spam_eval.sh output-file/group_x_model
!/u/cs451/bin/spam_eval.sh output-file/group_y_model
!/u/cs451/bin/spam_eval.sh output-file/britney_model

1-ROCA%: 17.26
1-ROCA%: 12.82
1-ROCA%: 19.09


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.